# Ejemplo de red con capa convolucional

En esta segunda parte de la auxiliar vamos a comparar el desempeño de una capa lineal + softmax con la arquitectura de CNN que se menciona en las diapositivas. Nuevamente esta parte es super práctica así que revisen el código y lean los comentarios.

In [ ]:
%%capture --no-stderr
!pip install --upgrade torchtext

In [ ]:
# Referencia https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
# Este ejemplo de clasificacion de texto lo saque de la referencia de arriba
# La idea es comparar la red convolucional con los resultados de una red
# feed forward simple para clasificar texto. Le hice algunas modificaciones
# pequennas, ya sea para simplificarlo o para mostrar alguna otra utilidad
# de pytorch
import os
import random
import torch
import torchtext
from torchtext.datasets import text_classification

# Setear seed para que los resultados sean replicables
torch.manual_seed(8888)
random.seed(8888)

# Cargar el dataset y cachearlo
if not os.path.isdir('./.data'):
    os.mkdir('./.data')

train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', vocab=None)


120000lines [00:05, 23554.77lines/s]
120000lines [00:11, 10676.09lines/s]
7600lines [00:00, 11008.10lines/s]


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Definir la primera arquitectura, esta es nuevamente una capa de embeddings
# y una capa lineal. Fijense en la funcion para inicializar los pesos. Esta
# comentada para no alterar la comparacion con la otra, pero la pueden
# descomentar y probar como cambian los resultados. Ademas les sirve como
# ejemplo para  saber como cambiar la inicializacion de los pesos para sus
# experimentos en la competencia 2
class TextSentimentLinear(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, mode="max")
        self.fc = nn.Linear(embed_dim, num_class)
        # self.init_weights() # probar agregando esto

    def init_weights(self): 
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.embedding.weight.data[train_dataset.get_vocab()["<pad>"]].zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)

In [ ]:
# Esta es una red con capa convolucional. La documentacion la pueden pilla
# aqui: https://pytorch.org/docs/stable/nn.html#conv1d
# Lo demas deberia ser identico a la red vista en clases
class TextSentimentConv1d(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=train_dataset.get_vocab()["<pad>"]
        )
        # El primer argumento corresponde a la cantidad de canales de entrada,
        # a diferencia de las imagenes que tienen 3 canales (RGB) nuestra frase
        # es un puro canal. El segundo argumento corresponde a los canales de
        # salida, que para este caso es la cantidad de clases que tenemos,
        # porque queremos tomar el tensor de salida, hacer maxpooling y usarlo
        # directo para clasificar. El tamanno del kernel, dado que queremos
        # hacer una convolucion para 3 paralabras, tiene que ser 3 x embed_dim
        # porque en la practica la convolucion se realiza sobre los embeddings
        # de la palabra y no la palabra misma. Finalmente el stride corresponde
        # a la cantidad de elementos que se `saltan` al avanzar la ventana.
        # Como no queremos hacer convoluciones con la mitad del embedding de una
        # y la mitad del embedding de otra, tenemos que saltarnos una cantidad
        # de elementos igual al tamanno de los vectores de embedding
        self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=num_class,
            kernel_size=3*embed_dim,
            stride=embed_dim
        )
    
    def forward(self, text):
        # embedded -> B, N, E
        embedded = self.embedding(text)
        # ahora hay que hacerle un flatten a las frases, es decir, concatenar
        # los vectores de embedding para hacer la convolucion. El 1 extra
        # en el metodo view es por un requisito de la capa convolucional, 
        # corresponse a la cantidad de `canales` (piensen en imagenes rgb),
        # pero como nuestra frase es un puro canal, ahi va un 1
        embedded = embedded.view(embedded.shape[0], 1, -1)
        z = F.relu(self.conv(embedded))
        # luego hacemos maxpooling, se usa el atributo .values porque la funcion
        # que hace saca el maximo, cuando se le pasa una dimension, retorna
        # tambien los indices en los que se encontraron los maximos.
        return z.max(dim=-1).values

In [ ]:
# Esta es la funcion que se usa para generar los batch a partir de una lista de 
# frases. Fijense el uso de la funcion de utilidad `pad_sequence` que realiza
# el padding necesario a cada frase para que queden todas del mismo largo.
def generate_batch(batch):
    labels = torch.tensor([entry[0] for entry in batch])
    sequences = [entry[1] for entry in batch]
    text = nn.utils.rnn.pad_sequence(
        sequences, batch_first=True, padding_value=train_dataset.get_vocab()["<pad>"]
    )
    return text, labels

In [ ]:
# Esta celda es lo de toda la vida, definir funciones para entrenar y evaluar
from torch.utils.data import DataLoader

def train_func(sub_train_, model):

    # Train the model
    train_loss ,train_acc = 0, 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for text, cls in data:
        optimizer.zero_grad()
        text, cls = text.to(device), cls.to(device)
        output = model(text)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_, model):
    loss, acc = 0, 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, cls in data:
        text, cls = text.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
# Esta celda y la siguiente entrenan los modelos para poder comparar el 
# desempenno. Todo esto deberia ser ya conocido por ustedes asi que no lo voy
# a comentar. Cualquier duda siempre estamos disponibles por el foro o telegram

import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 10
BATCH_SIZE = 128
EMBED_DIM = 32
LEARN_RATE = 3.0

VOCAB_SIZE = len(train_dataset.get_vocab())
NUM_CLASS = len(train_dataset.get_labels())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_len = int(len(train_dataset) * 0.85)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

model = TextSentimentLinear(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)

print(f"\nEntrenando modelo {model}\n")
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_, model)
    valid_loss, valid_acc = test(sub_valid_, model)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')


Entrenando modelo TextSentimentLinear(
  (embedding): EmbeddingBag(95812, 32, mode=max)
  (fc): Linear(in_features=32, out_features=4, bias=True)
)

Epoch: 1  | time in 0 minutes, 2 seconds
	Loss: 0.1200(train)	|	Acc: 68.5%(train)
	Loss: 0.0003(valid)	|	Acc: 86.2%(valid)
Epoch: 2  | time in 0 minutes, 2 seconds
	Loss: 0.0216(train)	|	Acc: 84.0%(train)
	Loss: 0.0007(valid)	|	Acc: 69.8%(valid)
Epoch: 3  | time in 0 minutes, 2 seconds
	Loss: 0.0144(train)	|	Acc: 87.0%(train)
	Loss: 0.0003(valid)	|	Acc: 83.6%(valid)
Epoch: 4  | time in 0 minutes, 2 seconds
	Loss: 0.0096(train)	|	Acc: 89.5%(train)
	Loss: 0.0003(valid)	|	Acc: 86.9%(valid)
Epoch: 5  | time in 0 minutes, 2 seconds
	Loss: 0.0069(train)	|	Acc: 91.4%(train)
	Loss: 0.0004(valid)	|	Acc: 83.1%(valid)
Epoch: 6  | time in 0 minutes, 2 seconds
	Loss: 0.0051(train)	|	Acc: 93.2%(train)
	Loss: 0.0002(valid)	|	Acc: 86.1%(valid)
Epoch: 7  | time in 0 minutes, 2 seconds
	Loss: 0.0034(train)	|	Acc: 94.5%(train)
	Loss: 0.0003(valid)	|	Acc: 86

In [ ]:
N_EPOCHS = 20
BATCH_SIZE = 128
EMBED_DIM = 32
LEARN_RATE = 3.0

model = TextSentimentConv1d(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)

print(f"\nEntrenando modelo {model}\n")
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_, model)
    valid_loss, valid_acc = test(sub_valid_, model)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')



Entrenando modelo TextSentimentConv1d(
  (embedding): Embedding(95812, 32, padding_idx=1)
  (conv): Conv1d(1, 4, kernel_size=(96,), stride=(32,))
)

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0117(train)	|	Acc: 57.7%(train)
	Loss: 0.0001(valid)	|	Acc: 69.2%(valid)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0071(train)	|	Acc: 72.3%(train)
	Loss: 0.0001(valid)	|	Acc: 75.5%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0059(train)	|	Acc: 76.9%(train)
	Loss: 0.0001(valid)	|	Acc: 77.1%(valid)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0053(train)	|	Acc: 79.2%(train)
	Loss: 0.0001(valid)	|	Acc: 80.0%(valid)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0049(train)	|	Acc: 80.4%(train)
	Loss: 0.0001(valid)	|	Acc: 79.8%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0046(train)	|	Acc: 81.6%(train)
	Loss: 0.0001(valid)	|	Acc: 81.7%(valid)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0044(train)	|	Acc: 82.4%(train)
	Loss: 0.0001(valid)	|	Acc: 81